In [ ]:
from z3 import *
import time
import matplotlib.pyplot as plt


def check(num_txs):
    NUM_ROWS = 1
    NUM_TXS = num_txs


    t_stats = [
        [(Int(f'k_{r_id}_{s_id}'), Int(f'v_{r_id}_{s_id}')) for s_id in range(0, NUM_TXS + 1)] for r_id in range(0, NUM_ROWS)
    ]

    s = Solver()
    s.set("timeout", 30000)

    s.add(And(t_stats[0][0][0] == 1, t_stats[0][0][1] == 0))
    s.add(And(t_stats[0][NUM_TXS][0] == 1, t_stats[0][NUM_TXS][1] == NUM_TXS))

    transition_stats = [
        [((Int(f'k_{r_id}_{tx_id}_b'), Int(f'v_{r_id}_{tx_id}_b')), (Int(f'k_{r_id}_{tx_id}_a'), Int(f'v_{r_id}_{tx_id}_a')))
            for tx_id in range(1, NUM_TXS + 1)] 
            for r_id in range(0, NUM_ROWS)
    ]

    for r_id in range(0, NUM_ROWS):
        for i in range(NUM_TXS):
            f_cond = Implies(transition_stats[r_id][i][0][1] == i, And(transition_stats[r_id][i][1][1] == i + 1, transition_stats[r_id][i][1][0] == transition_stats[r_id][i][0][0]))
            f_not_cond = Implies(transition_stats[r_id][i][0][1] != i, And(transition_stats[r_id][i][1][1] == transition_stats[r_id][i][0][1], transition_stats[r_id][i][1][0] == transition_stats[r_id][i][0][0]))
            s.add(And(f_cond, f_not_cond))

    def gen_formula_link_states(s_id, tx_id, ba_idx):
        ret_list = []
        for r_id in range(NUM_ROWS):
            ret_list.append(t_stats[r_id][s_id][0] == transition_stats[r_id][tx_id - 1][ba_idx][0])
            ret_list.append(t_stats[r_id][s_id][1] == transition_stats[r_id][tx_id - 1][ba_idx][1])
        return And(ret_list)

    for tx_id in range(1, NUM_TXS + 1):
        for s_id in range(0, NUM_TXS):
            s.add(And(Implies(gen_formula_link_states(s_id, tx_id, 0), gen_formula_link_states(s_id + 1, tx_id, 1))), 
                  Implies(gen_formula_link_states(s_id + 1, tx_id, 1), gen_formula_link_states(s_id, tx_id, 0)))


    f = Function('f', IntSort(), IntSort())

    for i in range(1, 1 + NUM_TXS):
        for j in range(1, 1 + NUM_TXS):
            s.add(Implies(f(i) == j, gen_formula_link_states(j, i, 1)))

    for i in range(1, 1 + NUM_TXS):
        s.add(And(f(i) >= 1, f(i) <= NUM_TXS))
        for j in range(1, i):
            s.add(f(i) != f(j))

    start_time = time.time()

    result = s.check()

    if result == sat:
        print("SAT")
        m = s.model()
        # print(m)
    else:
        print("UNSAT")

    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")
    return end_time - start_time


xs = [i + 1 for i in range(21)]
ys = [check(x) for x in xs]
plt.plot(xs, ys)
plt.show()

